In [1]:
!pip install requests beautifulsoup4 openai
!pip install requests beautifulsoup4 jina





   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.9/362.9 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.2/152.2 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opentelemetry-exporter-otlp-proto-grpc to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of o

In [9]:
import requests
from bs4 import BeautifulSoup
import json
import os


if not os.path.exists('scraped_content'):
    os.makedirs('scraped_content')

def scrape_links(url):
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        links = []
        for a_tag in soup.find_all('a', href=True):
            link = a_tag['href']
            if link.startswith('http'):
                links.append(link)
        return links
    except Exception as e:
        print(f"Error scraping {url}: {str(e)}")
        return []

def scrape_content(url):
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        content = soup.get_text(separator=' ', strip=True)
        return content
    except Exception as e:
        print(f"Error scraping content from {url}: {str(e)}")
        return ""

def save_to_json(data, filename):
    with open(filename, 'w') as f:
        json.dump(data, f, indent=4)

def main_scrape(url):
    links = scrape_links(url)
    data = []
    for link in links:
        content = scrape_content(link)
        if content:
            page_data = {
                "url": link,
                "content": content
            }
            data.append(page_data)
            save_to_json(page_data, f'scraped_content/{link.split("/")[-1]}.json')
    return data


url = 'https://replit.com'
all_data = main_scrape(url)
save_to_json(all_data, 'all_scraped_data.json')



In [12]:
!pip install --upgrade openai



In [13]:
import openai
import json
import os


openai.api_key = 'sk-McyJI4dQcEyYfabPZVO6ruqffLpZZ6mL5Ma9gOD85WT3BlbkFJN0TK00DA6kb1fE8FwNrw9j4Gh0sAG0JgmOAde6ZZAA'  # Replace with your OpenAI API key

def generate_questions(content):
    try:
        prompt = f"Generate 10 questions related to the following content:\n\n{content[:1000]}"
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
           messages=[
        {"role": "user", "content": "Generate a question about the content"}
    ],
            max_tokens=50,
            n=1,
            stop=None,
            temperature=0.5,
        )
        questions = response.choices[0].text.strip().split('\n')
        questions = [q.strip() for q in questions if q and len(q.strip()) <= 80]
        return questions[:10]
    except Exception as e:
        print(f"Error generating questions: {str(e)}")
        return []

def process_all_files():
    files = [f for f in os.listdir('scraped_content') if f.endswith('.json')]
    for file in files:
        with open(f'scraped_content/{file}', 'r') as f:
            data = json.load(f)
            questions = generate_questions(data['content'])
            data['questions'] = questions
            save_to_json(data, f'scraped_content/{file}')


process_all_files()


Error generating questions: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742

Error generating questions: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742

Error gene

In [14]:
import json
import os

def find_relevant_links(all_data, current_url):
    relevant_links = []
    for entry in all_data:
        if entry['url'] != current_url:
            relevant_links.append(entry['url'])
        if len(relevant_links) >= 5:
            break
    return relevant_links

def update_files_with_links(all_data):
    files = [f for f in os.listdir('scraped_content') if f.endswith('.json')]
    for file in files:
        with open(f'scraped_content/{file}', 'r') as f:
            data = json.load(f)
            relevant_links = find_relevant_links(all_data, data['url'])
            data['relevant_links'] = relevant_links
            save_to_json(data, f'scraped_content/{file}')


update_files_with_links(all_data)



In [15]:
import json
import os

def validate_json_files():
    files = [f for f in os.listdir('scraped_content') if f.endswith('.json')]
    for file in files:
        with open(f'scraped_content/{file}', 'r') as f:
            data = json.load(f)
            if len(data.get('questions', [])) != 10:
                print(f"Validation Error: {file} does not have 10 questions.")
            if len(data.get('relevant_links', [])) != 5:
                print(f"Validation Error: {file} does not have 5 relevant links.")
            for question in data.get('questions', []):
                if len(question) > 80:
                    print(f"Validation Error: A question in {file} exceeds 80 characters.")

def evaluate_performance():

    pass


validate_json_files()


Validation Error: login.json does not have 10 questions.
Validation Error: collaboration.json does not have 10 questions.
Validation Error: replit.json does not have 10 questions.
Validation Error: teams#join-the-beta.json does not have 10 questions.
Validation Error: signup.json does not have 10 questions.
Validation Error: ai.json does not have 10 questions.
Validation Error: cloud-development-environment.json does not have 10 questions.
Validation Error: us-student-dpa.json does not have 10 questions.
Validation Error: abuse-report.json does not have 10 questions.
Validation Error: blog.replit.com.json does not have 10 questions.
Validation Error: repl.it.json does not have 10 questions.
Validation Error: @replit.json does not have 10 questions.
Validation Error: teams-ga.json does not have 10 questions.
Validation Error: deployments.json does not have 10 questions.
Validation Error: .json does not have 10 questions.


In [16]:
{
    "url": "https://replit.com/teams",
    "content": "Full content of the webpage here",
    "questions": [
        "What is the main topic of the webpage?",
        "Who is the author?"
    ],
    "relevant_links": [
        "https://replit.com/pricing",
        "https://blog.replit.com/"
    ],
    "topics": [
        "Main topic of the webpage",
        "Author information"
    ]
}


{'url': 'https://replit.com/teams',
 'content': 'Full content of the webpage here',
 'questions': ['What is the main topic of the webpage?', 'Who is the author?'],
 'relevant_links': ['https://replit.com/pricing', 'https://blog.replit.com/'],
 'topics': ['Main topic of the webpage', 'Author information']}